In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# plt.style.use("dark_background")
import seaborn as sns

from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression



In [8]:
df_20180507073502 = pd.read_csv('../gfc/data/ICEWS/20180507073502.Release395_Factiva.csv')

In [9]:
df_20180507073502.head()

,Cameo Code,City,Country,Datetime,District,Event Date,Event ID,Event Sentence,Event Text,Headline,...,Source,Source Country,Source Name,Source Sectors,Story ID,Target Country,Target Name,Target Sectors,Top Cameo Code,Year
0,173,Jerusalem,Israel,2018-04-05,NaN,2018-04-05,ICEWS28760121,"Along the way, Olmert did not hesitate to atta...","Arrest, detain, or charge with legal action",The strike that spares Ehud Olmert,...,Factiva,Israel,Court Judge (Israel),"Government,Judicial",47073261,Israel,Ehud Olmert,"Ideological,Jews,Parties,(National) Major Part...",17,2018
1,20,Jerusalem,Israel,2018-04-13,NaN,2018-04-13,ICEWS28768305,"The event's organizer, Culture and Sport Minis...",Make an appeal or request,Hallelujah Israel,...,Factiva,Israel,Miri Regev,"Legislative / Parliamentary,Government,Unicame...",47090638,Honduras,Juan Orlando Hernández,"Executive,Ideological,Center Right,Unicameral,...",2,2018
2,90,Calgary,Canada,2018-04-17,NaN,2018-04-17,ICEWS28761359,CALGARY – Two years before provincial official...,Investigate,Calgary police refused calls from provincial w...,...,Factiva,Canada,Provincial Officials (Canada),"Local,Provincial,Government",47076007,Canada,Police (Canada),"Police,Government",9,2018
3,10,Stabroek,Guyana,2018-04-19,NaN,2018-04-19,ICEWS28751777,Minister of Finance Winston Jordan last Friday...,Make statement,Holder looking for candidates for new GuySuCo ...,...,Factiva,Guyana,Winston Jordan,"Government,Executive,Finance / Economy / Comme...",47056892,Guyana,Guyana Sugar Corporation,"State-Owned Agricultural,Government,State Owne...",1,2018
4,36,NaN,North Korea,2018-04-24,NaN,2018-04-24,ICEWS28749304,"First, North Korea's violently ruthless dictat...",Express intent to meet or negotiate,Before it's too late,...,Factiva,North Korea,Kim Jong-Un,"Executive Office,Government,Executive,(Nationa...",47052270,United States,Donald Trump,"Government,Executive,Executive Office",3,2018


In [10]:
!pip install neo4j-driver

    100% |████████████████████████████████| 194kB 1.5MB/s ta 0:00:01
  Running setup.py bdist_wheel for neo4j-driver ... done
  Stored in directory: /home/jayan/.cache/pip/wheels/5e/80/6c/38ec78c1e9869e2116ad012ae3cb38bb74521855749e980a7c
  Running setup.py bdist_wheel for neotime ... done
  Stored in directory: /home/jayan/.cache/pip/wheels/19/1c/63/e1777cae4998cb32c5ae5bb1901def45cc94de3abff4d7053d
Successfully built neo4j-driver neotime
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [11]:
!python -m pip install --upgrade pip

    100% |████████████████████████████████| 1.3MB 624kB/s ta 0:00:01
  Found existing installation: pip 9.0.1
    Uninstalling pip-9.0.1:
      Successfully uninstalled pip-9.0.1


In [14]:
from neo4j.v1 import GraphDatabase


In [21]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "capgemini"))

def add_friends(tx, name, friend_name):
    tx.run("MERGE (a:Person {name: $name}) "
           "MERGE (a)-[:KNOWS]->(friend:Person {name: $friend_name})",
           name=name, friend_name=friend_name)

def print_friends(tx, name):
    for record in tx.run("MATCH (a:Person)-[:KNOWS]->(friend) WHERE a.name = $name "
                         "RETURN friend.name ORDER BY friend.name", name=name):
        print(record["friend.name"])

with driver.session() as session:
    session.write_transaction(add_friends, "Arthur", "Guinevere")
    session.write_transaction(add_friends, "Arthur", "Lancelot")
    session.write_transaction(add_friends, "Arthur", "Merlin")
    session.read_transaction(print_friends, "Arthur")

Guinevere
Lancelot
Merlin
